<a href="https://colab.research.google.com/github/GuoZhuyin/zhuyinasia/blob/main/DeepLearning/FarmlandSubmit_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 輸出的數值每20個進行判斷，為14個作物類別的哪一個

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
class_to_index={'banana': 0, 'bareland': 1, 'building': 2, 'carrot': 3, 'corn': 4,
    'dragonfruit': 5, 'garlic': 6, 'guava': 7, 'mountain': 8, 'peanut': 9, 'pineapple': 10,
    'pumpkin': 11, 'rice': 12, 'sky': 13, 'soybean': 14, 'sugarcane': 15, 'tomato': 16}
index_to_class = {v: k for k, v in class_to_index.items()}

In [ ]:
df_labels = pd.read_csv('../submit/submit7.0_Team855.csv')
colnames0=[f'f{x}' for x in range(17)]
colnames1=[f'f{17+x}' for x in range(17)]
colnames2=[f'f{34+x}' for x in range(17)]

df_predict0 = pd.read_csv('../output/TEAM_855_6.10_output.csv',header=None, names=colnames0)
df_predict1 = pd.read_csv('../output/TEAM_855_7.0_output.csv',header=None, names=colnames1)
df_predict2 = pd.read_csv('../output/TEAM_855_9.0_output.csv',header=None, names=colnames2)

輸入由predictoutput_6輸出的csv黨

fbl_pt1_TEAM_855_6.10.pth模型測試結果輸出為TEAM_855_6.10_output.csv

Effi_B0_TEAM_855_7.0.pth模型測試結果輸出為TEAM_855_7.0_output.csv

Effi_B2_TEAM_855_9.0.pth模型測試結果輸出為TEAM_855_9.0_output.csv

In [ ]:
X = pd.concat([df_predict0, df_predict1, df_predict2], axis=1, join='inner')
X.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
y=[0 for i in range(len(df_predict0))]
for index, row in df_labels.iterrows():
  label = row[1]
  classIndex = class_to_index[label]
  for ii in range(index*20, (index+1)*20):
      y[ii] = classIndex
Y = pd.DataFrame(y, columns =['label'])

In [ ]:
len(X)

In [ ]:
Y.head()

In [ ]:
len(Y)

In [ ]:
import time
import xgboost as xgb
model = xgb.XGBClassifier(n_jobs = 16,objective='multi:softmax', use_label_encoder = False)

start = time.time()
modelfit = model.fit(X, Y, eval_metric='logloss')
fittingTime = time.time() - start

In [ ]:
y_pred = model.predict(X)
judgeClass = [round(value) for value in y_pred]

In [ ]:
def argmax(lst):
    return max(range(len(lst)), key=lst.__getitem__)

In [ ]:
def accumulate_blocks(stats, ii):
  if ii ==2 or ii==8 or ii==13:
    stats[ii] +=-1
    return
  elif ii ==1 or ii==15 or ii==0:
    stats[ii] +=1
    return
  stats[ii] +=2
  return

In [ ]:
def predict(ii):
  stats = [0 for i in range(17)]
  for i in range(20):
    index = ii*20+i
    dets = judgeClass[index]
    accumulate_blocks(stats, dets)
  judge = argmax(stats)
  landtype = index_to_class[judge]
  return landtype

In [ ]:
answer = []
correct_count =0
for index, row in df_labels.iterrows():
  filename = row[0]
  predicted_label = predict(index)
  answer.append(predicted_label)

In [ ]:
df_result = pd.DataFrame(df_labels['image_filename'])
df_result['label'] = pd.DataFrame(answer) 

In [ ]:
df_result.head()

In [ ]:
df_result.to_csv("submit_X2.csv", index=False )

對三個輸出結果進行XGBOOST輸出submit_X2.csv

In [ ]:
df_labels = pd.read_csv('submit7.0_Team855.csv')

In [ ]:
correct_count =0
for index, row in df_labels.iterrows():
  filename = row[0]
  label = row[1]
  predicted_label = answer[index]
  if predicted_label == label:
    correct_count += 1
  #else:
  #  print(filename, label, predicted_label)
accuracy = correct_count/len(df_labels) 
print(f"{accuracy:.3f}")

https://aidea-web.tw/topic/93c8c26b-0e96-44bc-9a53-1c96353ad340

評估方式採用Weighted-Precision(WP), 且各類別F1-score需大於0.7
$$WP={\sum_{i=1}^{NumofClass} (Precision_i \times (TP_i + FN_i)) \over TotalImageCount}$$

$$\text{F1-score}= 2 \times {(Precision \times Recall) \over (Precision + Recall) }$$

$$\text{Recall}= {TP \over TP + FN }$$

$$\text{Precision}= {TP \over TP + FP }$$

In [ ]:
landtypes=['banana', 'bareland', 'carrot', 'corn', 'dragonfruit', 'garlic', 'guava', 'peanut', 'pineapple', 'pumpkin', 'rice', 'soybean', 'sugarcane', 'tomato']
print(len(landtypes))

In [ ]:
TP = {lt:0 for lt in landtypes}
FN = {lt:0 for lt in landtypes}
FP = {lt:0 for lt in landtypes}
for index, row in df_labels.iterrows():
  label = row[1]
  predicted_label = answer[index]
  if predicted_label == label:
     TP[label] +=1
  else:
      FP[label] +=1
      FN[predicted_label] +=1

In [ ]:
for lt in landtypes:
  if TP[lt]+FN[lt] >0:
    recall = TP[lt]/(TP[lt]+FN[lt])
  else:
    recall = 0
  if TP[lt]+FP[lt] >0:
    precision = TP[lt]/(TP[lt]+FP[lt])
  else:
    precision = 0
  f1_score = 2*precision*recall/(precision+recall)
  print(f"{lt:12}:precision={precision:.2f}:recall={recall:.2f}:f1-score={f1_score:.2f}")

In [ ]:
All=len(df_labels)
score = 0.0
for lt in landtypes:
  if TP[lt]+FN[lt] >0:
    recall = TP[lt]/(TP[lt]+FN[lt])
  else:
    recall = 0
  if TP[lt]+FP[lt] >0:
    precision = TP[lt]/(TP[lt]+FP[lt])
  else:
    precision = 0
  score += precision*(TP[lt]+FN[lt])

Weighted_Precision = score/All
print(f"Weighted_Precision={Weighted_Precision:.3f}")